#  <center> Telecons + DLmath (CNN+ LSTM) </center>

## 1. Data Preparation

In [1]:
import os
import pandas as pd
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import cv2
import numpy as np
from torch.utils.data import Dataset
import random

In [2]:
# 데이터 폴더 경로
data_folder = '/home/work/DLmath/Seulbin/Telecons/mydata/C'

In [3]:
# 클래스 통합 매핑
label_map = {
    "C1": "Normal",  # 정상
    "C3": "Normal",  # 정상
    "C4": "Normal",  # 정상
    "C2": "BedExit", # 침대 이탈
    "C5": "BedExit"  # 침대 이탈
}

In [4]:
# 레이블 저장용 리스트
labels = []

In [5]:
for file_name in os.listdir(data_folder):
    if file_name.endswith(('.mp4', '.avi', '.mkv')):  # 비디오 파일만 처리
        # 원본 레이블 추출
        original_label = file_name.split('_')[0]  # 파일 이름의 첫 번째 부분을 레이블로 사용
        
        # 통합 레이블 매핑
        if original_label in label_map:
            unified_label = label_map[original_label]
            labels.append({
                'FilePath': os.path.join(data_folder, file_name),
                'Label': unified_label
            })

In [6]:
# 데이터프레임으로 변환 후 CSV로 저장
df = pd.DataFrame(labels)
df.to_csv('video_labels.csv', index=False)

## 2. Define the PyTorch Dataset 

In [7]:
class VideoDataset(Dataset):
    def __init__(self, csv_file, num_frames=6, transform=None):
        self.data = pd.read_csv(csv_file)
        self.num_frames = num_frames
        self.transform = transform
        self.label_map = {label: idx for idx, label in enumerate(self.data['Label'].unique())}

    def extract_frames(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = max(1, total_frames // self.num_frames)
        
        for i in range(self.num_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (256, 256))
                frame = frame / 255.0  # Normalize
                if self.transform:
                    frame = self.transform(frame)
                frames.append(frame)

        cap.release()
        # 프레임 수가 부족한 경우 마지막 프레임 복제 또는 검정 화면 추가
        while len(frames) < self.num_frames:
            if len(frames) > 0:
                frames.append(frames[-1])  # 마지막 프레임 복제
            else:
                frames.append(np.zeros((256, 256, 3), dtype=np.float32))  # 검정 화면 추가

        # 초과된 프레임은 자르기
        frames = frames[:self.num_frames]
        
        return torch.stack(frames)  # Tensor로 반환

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video_path = self.data.iloc[idx]['FilePath']
        label = self.data.iloc[idx]['Label']
        frames = self.extract_frames(video_path)
        label_idx = self.label_map[label]
        return frames.permute(0, 3, 1, 2), torch.tensor(label_idx)

## 3. CNN-LSTM Model Design

In [8]:
## CNN+LSTM 모델
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        # x: (batch, time_steps, channels, height, width)
        batch_size, time_steps, c, h, w = x.size()
        x = x.view(-1, c, h, w)  # 배치와 타임스텝 결합
        x = self.module(x)            
            
        out_c, out_h, out_w = x.size(1), x.size(2), x.size(3)
        x = x.view(batch_size, time_steps, out_c, out_h, out_w)  # 배치와 타임스텝 복원
        return x


class CNNLSTMModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNLSTMModel, self).__init__()
        
        # CNN Layers
        self.cnn = nn.Sequential(
            TimeDistributed(nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)),
            TimeDistributed(nn.BatchNorm2d(16)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(4)), # 256 -> 64
            TimeDistributed(nn.Dropout(0.3)),
            
            TimeDistributed(nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)),
            TimeDistributed(nn.BatchNorm2d(32)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(4)), # 56 -> 16
            TimeDistributed(nn.Dropout(0.3)),
            
            TimeDistributed(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)),
            TimeDistributed(nn.BatchNorm2d(64)),
            TimeDistributed(nn.ReLU()),
            TimeDistributed(nn.MaxPool2d(4)), # 16 -> 4
            TimeDistributed(nn.Dropout(0.3)),
        )
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size=64*4*4, hidden_size=32, batch_first=True) 
        
        # Fully Connected Layer
        self.fc = nn.Linear(32, num_classes)
        
    def forward(self, x):
        batch_size, time_steps, c, h, w = x.size()
        x = x.permute(0, 1, 3, 2, 4)
        x = self.cnn(x) #CNN Forward # Shape: (batch_size, time_steps, 64, 7, 7)
        x = x.contiguous().view(batch_size, time_steps, -1)  # Shape: (batch_size, time_steps, 64*7*7)
        _, (hidden, _) = self.lstm(x) # Shape: hidden: (1, batch_size, 16)
        x = hidden[-1]  # Shape: (batch_size, 16)
        x = self.fc(x) # Shape: (batch_size, num_classes)
        return x

### Early stopping

In [9]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0, path='checkpoint.pt'):
        """
            patience (int): 성능이 개선되지 않을 때 기다리는 epoch 수.
            verbose (bool): True일 경우, 성능 개선 시 메시지를 출력.
            delta (float): 성능 향상으로 간주할 최소 변화량.
            path (str): 체크포인트를 저장할 경로.
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """검증 손실이 감소할 때 모델을 저장합니다."""
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## 4. Data Loader

In [10]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


# 데이터 분할
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)
train_data.to_csv('train_labels.csv', index=False)
val_data.to_csv('val_labels.csv', index=False)

# 데이터셋 초기화
train_dataset = VideoDataset(csv_file='train_labels.csv')
val_dataset = VideoDataset(csv_file='val_labels.csv')

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # 배치 크기를 4로 설정
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)  # 검증 데이터는 섞지 않음

## 5. Training Loop

In [11]:
from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 클래스 가중치 계산
class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(train_data['Label']), 
    y=df['Label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


# 모델 초기화
num_classes = len(df['Label'].unique())
model = CNNLSTMModel(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001) 

# EarlyStopping 객체 생성
early_stopping = EarlyStopping(patience=10, verbose=True, path='best_model.pt')

# Scheduler 정의
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# 학습 및 검증 손실/정확도 기록용 리스트
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for frames, labels in train_loader:
        frames, labels = frames.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(100. * correct / total)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%")
    
    # validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for frames, labels in val_loader:
            frames, labels = frames.to(device), labels.to(device)
            outputs = model(frames)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(100. * correct / total)

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%")
    
    # 학습률 스케줄러 업데이트
    scheduler.step(val_loss / len(val_loader))

    # EarlyStopping 호출
    early_stopping(val_loss, model)

    # Early Stopping 체크
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Early Stopping 이후 최적 모델 로드
if early_stopping.early_stop:
    model.load_state_dict(torch.load('best_model.pt'))

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

## save the model

In [ ]:
import datetime as dt

# Validation Loss와 Accuracy 계산
val_loss_avg = val_loss / len(val_loader)
val_accuracy = 100. * correct / total

# 현재 날짜 및 시간 포맷 지정
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time = dt.datetime.now().strftime(date_time_format)

# 파일 이름 정의
model_file_name = f"cnn_lstm_model__Date_Time_{current_date_time}__Loss_{val_loss_avg:.4f}__Accuracy_{val_accuracy:.2f}.pth"

# 모델 저장
torch.save(model.state_dict(), model_file_name)
print(f"Model saved as {model_file_name}")

## 시각화

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(train_values, val_values, metric_name):
    """
        train_values: 훈련 데이터 값 리스트
        val_values: 검증 데이터 값 리스트
        metric_name: 지표 이름 (e.g., "Loss", "Accuracy")
    """
    epochs = range(1, len(train_values) + 1)
    plt.plot(epochs, train_values, 'b-', label=f'Train {metric_name}')
    plt.plot(epochs, val_values, 'r-', label=f'Validation {metric_name}')
    plt.title(f'Train and Validation {metric_name}')
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.legend()
    plt.show()

In [ ]:
# 손실 시각화
plot_metrics(train_losses, val_losses, "Loss")

# 정확도 시각화
plot_metrics(train_accuracies, val_accuracies, "Accuracy")